In [9]:
import pandas as pd
df = pd.read_csv("pima.dat")
df.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Class
0,6,148,72,35,0,33.6,0.627,50,positive
1,1,85,66,29,0,26.6,0.351,31,negative
2,8,183,64,0,0,23.3,0.672,32,positive
3,1,89,66,23,94,28.1,0.167,21,negative
4,0,137,40,35,168,43.1,2.288,33,positive


In [10]:
df['Class'] = df['Class'].replace({'negative' : 0, 'positive' : 1})

In [11]:
# Все типы данных подобраны правильно
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Preg    768 non-null    int64  
 1   Plas    768 non-null    int64  
 2   Pres    768 non-null    int64  
 3   Skin    768 non-null    int64  
 4   Insu    768 non-null    int64  
 5   Mass    768 non-null    float64
 6   Pedi    768 non-null    float64
 7   Age     768 non-null    int64  
 8   Class   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [13]:
#Стандартизация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(df.iloc[:, :-1])
y = df["Class"]
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [14]:
#Делим выборку на обучающую и валидационную и тестовую
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(x, y, 
                                                    test_size=0.5, 
                                                    random_state=2, stratify=y)

x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, 
                                                    test_size=0.4, 
                                                    random_state=2, stratify=y_temp)
x_train.shape[0], x_test.shape[0], x_val.shape[0]

(384, 230, 154)

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score

ccp_alpha  = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.2, 0.8]
criterion = ['entropy', 'gini']

df_temp = []
for crit in criterion:
    for alpha in ccp_alpha:
        temp = []
        temp.append(alpha)
        temp.append(crit)
        tree = DecisionTreeClassifier(ccp_alpha=alpha, criterion=crit)
        tree.fit(x_train, y_train)
        y_pred = tree.predict(x_test)
        temp.append(roc_auc_score(y_test, y_pred,  average='macro'))
        df_temp.append(temp)

tree_scores = pd.DataFrame(df_temp)
tree_scores.columns = ['alpha', 'criterion', 'roc_auc_score']
tree_scores.sort_values(by=[ 'roc_auc_score'], ascending=[ False])
# Best params: Criterion : gini, Alpha : 0.015

,alpha,criterion,roc_auc_score
11,0.015,gini,0.692917
4,0.025,entropy,0.692917
5,0.030,entropy,0.692917
6,0.035,entropy,0.692917
13,0.025,gini,0.692917
12,0.020,gini,0.692917
15,0.035,gini,0.679167
14,0.030,gini,0.679167
3,0.020,entropy,0.662083
10,0.010,gini,0.662083


In [19]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.svm import SVC
from tqdm import tqdm

krls  = ['linear', 'rbf', 'sigmoid'] # 'poly'
gammas = ['scale', 'auto', 1, 10, 0.1, 20, 0.01]
coef0s = [0, 1, 2, 5, 10, 0.1, 0.01]
degrees = [2, 3, 4, 5]
cs = [0.001, 0.01, 0.1, 0.5, 1, 2, 10, 20]

df_temp = []
for c in tqdm(cs):
    print(c)
    for krl in krls:
        print(krl)
        for gamma in gammas:
            for coef0 in coef0s:
                for degree in degrees:
                    temp = []
                    temp.append(krl)
                    temp.append(gamma)
                    temp.append(coef0)
                    temp.append(degree)
                    temp.append(c)
                    clf = SVC(kernel=krl, gamma=gamma, coef0=coef0, degree=degree, C=c, probability=True, cache_size=6000)
                    clf.fit(x_train, y_train)
                    y_pred = clf.predict(x_test)
                    temp.append(roc_auc_score(y_test, y_pred, average='macro'))
                    df_temp.append(temp)

svc_scores = pd.DataFrame(df_temp)
svc_scores.columns = ['kernel', 'gamma', "coef0", "degree", "C" , 'roc_auc_score']
svc_scores.sort_values(by=['roc_auc_score'], ascending=False)
# Best params: kernel = rbf, gamma = 0.01, coef0 = 0.01, degree = 4, C = 10

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
  0%|          | 0/8 [00:00<?, ?it/s]

0.001
linear
rbf
sigmoid


 12%|█▎        | 1/8 [00:17<02:02, 17.44s/it]

0.01
linear
rbf
sigmoid


 25%|██▌       | 2/8 [00:34<01:42, 17.04s/it]

...

10
linear
rbf
sigmoid


 88%|████████▊ | 7/8 [02:09<00:20, 20.76s/it]

20
linear
rbf
sigmoid


100%|██████████| 8/8 [02:52<00:00, 21.54s/it]


,kernel,gamma,coef0,degree,C,roc_auc_score
3918,rbf,0.01,0.01,4,10.0,0.750833
3904,rbf,0.01,5.00,2,10.0,0.750833
3919,rbf,0.01,0.01,5,10.0,0.750833
3892,rbf,0.01,0.00,2,10.0,0.750833
3893,rbf,0.01,0.00,3,10.0,0.750833
...,...,...,...,...,...,...
3786,rbf,1,1.00,4,10.0,0.496250
4368,rbf,1,0.00,2,20.0,0.496250
4370,rbf,1,0.00,4,20.0,0.496250
4371,rbf,1,0.00,5,20.0,0.496250


In [20]:
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

max_features = range(2, 9)
n_trees  = [50, 100, 150, 200, 300]
criterion = ['entropy', 'gini']

df_temp = []
for n_tree in tqdm(n_trees):
    for crit in criterion:
        for max_f in max_features:
            temp = []
            temp.append(n_tree)
            temp.append(max_f)
            temp.append(crit)
            clf = RandomForestClassifier(n_estimators=n_tree, criterion=crit, max_features=max_f, random_state=42)
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            temp.append(roc_auc_score(y_test, y_pred,  average='macro'))
            df_temp.append(temp)

forest_scores = pd.DataFrame(df_temp)
forest_scores.columns = ['n_trees', "max_features", 'criterion', 'roc_auc_score']
forest_scores.sort_values(by=['roc_auc_score'], ascending=[ False])
# Best params: Criterion : gini, n_estimators : 50, max_features : 7

100%|██████████| 5/5 [00:10<00:00,  2.17s/it]


,n_trees,max_features,criterion,roc_auc_score
12,50,7,gini,0.727500
10,50,5,gini,0.727083
37,150,4,gini,0.726667
9,50,4,gini,0.726250
7,50,2,gini,0.720833
...,...,...,...,...
28,150,2,entropy,0.673750
26,100,7,gini,0.673333
29,150,3,entropy,0.667917
41,150,8,gini,0.667500


In [21]:
import numpy as np

first_class = df.groupby("Class").size()[0]
second_class = df.groupby("Class").size()[1]
df_temp = df[df['Class'] == 1]
df_temp.reset_index(inplace=True)
df_oversampling = df
for _ in range(first_class - second_class):
    index = np.random.randint(0, second_class)
    df_oversampling = df_oversampling.append(df_temp.loc[[index]])
df_oversampling= df_oversampling.drop('index', axis=1)

In [22]:
df_oversampling.groupby("Class").size()

Class
0    500
1    500
dtype: int64

In [23]:
scaler = StandardScaler()
x = scaler.fit_transform(df_oversampling.iloc[:, :-1])
y = df_oversampling["Class"]

In [24]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, 
                                                    test_size=0.5, 
                                                    random_state=2, stratify=y)

x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, 
                                                    test_size=0.4, 
                                                    random_state=2, stratify=y_temp)
x_train.shape[0], x_test.shape[0], x_val.shape[0]

(500, 300, 200)

In [25]:
ccp_alpha  = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.2, 0.8]
criterion = ['entropy', 'gini']

df_temp = []
for crit in criterion:
    for alpha in ccp_alpha:
        temp = []
        temp.append(alpha)
        temp.append(crit)
        tree = DecisionTreeClassifier(ccp_alpha=alpha, criterion=crit)
        tree.fit(x_train, y_train)
        y_pred = tree.predict(x_test)
        temp.append(roc_auc_score(y_test, y_pred,  average='macro'))
        df_temp.append(temp)

tree_scores = pd.DataFrame(df_temp)
tree_scores.columns = ['alpha', 'criterion', 'roc_auc_score']
tree_scores.sort_values(by=['roc_auc_score'], ascending=[ False])

,alpha,criterion,roc_auc_score
0,0.005,entropy,0.756667
2,0.015,entropy,0.746667
4,0.025,entropy,0.740000
15,0.035,gini,0.733333
5,0.030,entropy,0.733333
6,0.035,entropy,0.733333
14,0.030,gini,0.733333
13,0.025,gini,0.733333
10,0.010,gini,0.730000
9,0.005,gini,0.730000


In [26]:
krls  = ['linear', 'rbf', 'sigmoid'] # 'poly'
gammas = ['scale', 'auto', 1, 10, 0.1, 20, 0.01]
coef0s = [0, 1, 2, 5, 10, 0.1, 0.01]
degrees = [2, 3, 4, 5]
cs = [0.001, 0.01, 0.1, 0.5, 1, 2, 10, 20]

df_temp = []
for c in tqdm(cs):
    print(c)
    for krl in krls:
        print(krl)
        for gamma in gammas:
            for coef0 in coef0s:
                for degree in degrees:
                    temp = []
                    temp.append(krl)
                    temp.append(gamma)
                    temp.append(coef0)
                    temp.append(degree)
                    temp.append(c)
                    clf = SVC(kernel=krl, gamma=gamma, coef0=coef0, degree=degree, C=c, probability=True, cache_size=6000)
                    clf.fit(x_train, y_train)
                    y_pred = clf.predict(x_test)
                    temp.append(roc_auc_score(y_test, y_pred, average='macro'))
                    df_temp.append(temp)

svc_scores = pd.DataFrame(df_temp)
svc_scores.columns = ['kernel', 'gamma', "coef0", "degree", "C" , 'roc_auc_score']
svc_scores.sort_values(by=['roc_auc_score'], ascending=False)
# Best params: kernel = rbf, gamma = auto, coef0 = 10, degree = 2 C = 0.5

  0%|          | 0/8 [00:00<?, ?it/s]

0.001
linear
rbf
sigmoid


 12%|█▎        | 1/8 [00:25<03:00, 25.72s/it]

0.01
linear
rbf
sigmoid


 25%|██▌       | 2/8 [00:50<02:30, 25.11s/it]

...

10
linear
rbf
sigmoid


 88%|████████▊ | 7/8 [03:06<00:29, 29.99s/it]

20
linear
rbf
sigmoid


100%|██████████| 8/8 [04:00<00:00, 30.12s/it]


,kernel,gamma,coef0,degree,C,roc_auc_score
2004,rbf,auto,10.0,2,0.5,0.803333
2006,rbf,auto,10.0,4,0.5,0.803333
1989,rbf,auto,0.0,3,0.5,0.803333
1990,rbf,auto,0.0,4,0.5,0.803333
1991,rbf,auto,0.0,5,0.5,0.803333
...,...,...,...,...,...,...
2229,sigmoid,1,10.0,3,0.5,0.490000
2819,sigmoid,1,10.0,5,1.0,0.486667
2818,sigmoid,1,10.0,4,1.0,0.486667
2817,sigmoid,1,10.0,3,1.0,0.486667


In [27]:
max_features = range(2, 9)
n_trees  = [50, 100, 150, 200, 300]
criterion = ['entropy', 'gini']

df_temp = []
for n_tree in tqdm(n_trees):
    for crit in criterion:
        for max_f in max_features:
            temp = []
            temp.append(n_tree)
            temp.append(max_f)
            temp.append(crit)
            clf = RandomForestClassifier(n_estimators=n_tree, criterion=crit, max_features=max_f, random_state=42)
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            temp.append(roc_auc_score(y_test, y_pred,  average='macro'))
            df_temp.append(temp)

forest_scores = pd.DataFrame(df_temp)
forest_scores.columns = ['n_trees', "max_features", 'criterion', 'roc_auc_score']
forest_scores.sort_values(by=['roc_auc_score'], ascending=[ False])
# Best params: Criterion : 'entropy', n_estimators : 200, max_features : 4

100%|██████████| 5/5 [00:13<00:00,  2.61s/it]


,n_trees,max_features,criterion,roc_auc_score
44,200,4,entropy,0.843333
16,100,4,entropy,0.843333
30,150,4,entropy,0.840000
32,150,6,entropy,0.836667
28,150,2,entropy,0.833333
...,...,...,...,...
7,50,2,gini,0.800000
13,50,8,gini,0.796667
10,50,5,gini,0.796667
3,50,5,entropy,0.793333


In [ ]:
#Before oversampling 

In [28]:
tree = DecisionTreeClassifier(ccp_alpha=0.015, criterion='gini')
tree.fit(x_train, y_train)
y_pred = tree.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.695


In [29]:
clf = SVC(kernel="rbf", gamma=0.01, coef0=0.01, degree=4, C=10, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)
print(roc_auc_score(y_val, y_pred, average='macro'))

0.745


In [30]:
clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_features=7, random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(roc_auc_score(y_test, y_pred, average='macro'))

0.7933333333333332


In [ ]:
#After enrichmet

In [31]:
tree = DecisionTreeClassifier(ccp_alpha=0.05, criterion='entropy')
tree.fit(x_train, y_train)
y_pred = tree.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.665


In [32]:
clf = SVC(kernel = 'rbf' , gamma = 'auto', coef0 = 10, degree = 2, C = 0.5, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)
print(roc_auc_score(y_val, y_pred, average='macro'))

0.79


In [33]:
clf = RandomForestClassifier(n_estimators=200, criterion='entropy', max_features=4, random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(roc_auc_score(y_test, y_pred, average='macro'))

0.8433333333333333
